In [ ]:
%matplotlib inline

In [ ]:
import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import sklearn.model_selection as sms
import sklearn.cross_validation as scv
import sklearn.metrics as skm
import sklearn.ensemble as ske

## t1. Random Forests

In [ ]:
df = pd.read_csv("./abalone.csv")

In [ ]:
def sex_2_int(x):
    return 1 if x == 'M' else (-1 if x == 'F' else 0)

def prep_df(_df):
    _z = _df.assign(Sex = _df.Sex.apply(sex_2_int))
    return _z

In [ ]:
df  = prep_df(df)

X = df.loc[:, df.columns[0:-1]].values
y = df.loc[:, df.columns[-1]].values

In [ ]:
def score(cl, _X, _y) -> float:
    cv = scv.KFold(_X.shape[0], n_folds=5, random_state=1, shuffle=True)    
    return np.mean(ms.cross_val_score(cl, _X, _y, cv=cv,  scoring='r2', n_jobs=4))

In [ ]:
n_est = range(1, 60)
scores = [score(ske.RandomForestRegressor(n_estimators=n, n_jobs=4), X, y) for n in tqdm.tqdm_notebook(n_est)] 

In [ ]:
with open("t1.txt", 'w') as f:
    f.write( "%d" % [(n,s) for n, s in zip(n_est, scores) if s > 0.52][0][0])

In [ ]:
pl.plot(n_est, scores)

## t2. GB

In [ ]:
df = pd.read_csv("/mnt/coursera/gbm-data.csv")

In [ ]:
y = df.loc[:, df.columns[0]].values
X = df.loc[:, df.columns[1:]].values

In [ ]:
def tt(X, y):
    return sms.train_test_split(X, y, test_size=0.8,  random_state=241)

In [ ]:
Xtr, Xte, ytr, yte = tt(X,y)

In [ ]:
def get_cl(lr: float):
    return ske.GradientBoostingClassifier(learning_rate=lr, n_estimators=250, verbose=True, random_state=241)

In [ ]:
cl = get_cl(0.2).fit(Xtr, ytr)
pass

In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [ ]:
def loss(y, y_true = yte):
    y_pred = sigmoid(y)
    ll = skm.log_loss(y_true=y_true, y_pred=y_pred)
    return ll

z = [(i, loss(y_val)) for i, y_val in enumerate(cl.staged_decision_function(Xte))]
ztr = [(i, loss(y_val, y_true=ytr)) for i, y_val in enumerate(cl.staged_decision_function(Xtr))]

a = min(z, key=lambda x: x[1])

In [ ]:
with open("t2.txt", "w") as f:
    f.write("%.2f %d" % (a[1], a[0]))

In [ ]:
with open("t3.txt", "w") as f:
    f.write("%s" % "overfitting")

In [ ]:
it = [x[0] for x in z]
ll = [x[1] for x in z]
pl.plot(it,ll)

In [ ]:
rf = ske.RandomForestClassifier(n_estimators=a[0], random_state=241).fit(Xtr, ytr)
rf_score = skm.log_loss(yte, rf.predict_proba(Xte)[:,1])

In [ ]:
with open("t4.txt", "w") as f:
    f.write("%.2f" % rf_score)